In [ ]:
# No words, just work => Goal: TOP #1 🍋

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Data Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
sns.set_theme(style='whitegrid', palette='viridis')

# Natural Language Processing | Machine Learning models
import spacy
import string
import gensim
import operator
import re
from tqdm import tqdm

# Ignore any warnings: chill..
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_train = pd.read_csv('/kaggle/input/nitec-epir-dataset/epir_train/articles.csv')
df_test = pd.read_csv('/kaggle/input/nitec-epir-dataset/epir_test.csv')

In [ ]:
print(len(df_train[df_train['sys_lang'].isin(['en', 'ru', 'qq', 'kk'])]))
print('Kazakh:', len(df_train[df_train['sys_lang'] == 'kk']))
print('Kazakh(Latin):', len(df_train[df_train['sys_lang'] == 'qq']))
print('Russian:', len(df_train[df_train['sys_lang'] == 'ru']))
print('English:', len(df_train[df_train['sys_lang'] == 'en']))

In [ ]:
df_train.head(10)

In [ ]:
# Training Datasets
df_articles = pd.read_csv('/kaggle/input/nitec-epir-dataset/epir_train/articles.csv')
df_lifes = pd.read_csv('/kaggle/input/nitec-epir-dataset/epir_train/life_situations.csv')
df_news = pd.read_csv('/kaggle/input/nitec-epir-dataset/epir_train/news.csv')
df_services = pd.read_csv('/kaggle/input/nitec-epir-dataset/epir_train/services.csv')

# Testing Datasets
df_test = pd.read_csv('/kaggle/input/nitec-epir-dataset/epir_test.csv')
df_subsample = pd.read_csv('/kaggle/input/nitec-epir-dataset/epir_sample_submission.csv')

In [ ]:
df_lifes.drop(columns=['subid'], inplace=True)
df_lifes.rename(columns={'URL': 'url'}, inplace=True)

def df_generalize(df, category):
    if 'content' in df.columns:
        df.rename(columns={'content': 'x'}, inplace=True)
    drop_columns = df.columns[3:-1]
    df['content'] = df.iloc[:, 3:-1].apply(lambda row: ' '.join(map(str, row)), axis=1)
    df.drop(columns=drop_columns, inplace=True)
    df.drop(['id'], axis=1, inplace=True)
    df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
    df = df.assign(**{'category': category})
    return df

In [ ]:
df_news = df_generalize(df_news, 'news')
df_lifes = df_generalize(df_lifes, 'lifes')
df_articles = df_generalize(df_articles, 'articles')
df_services = df_generalize(df_services, 'services')

In [ ]:
df_train = pd.concat([df_news, df_lifes, df_articles, df_services])

In [ ]:
df_train.head(10)

In [ ]:
df_train.dropna(inplace=True)

In [ ]:
df_train = df_train[df_train['sys_lang'].isin(['en', 'ru', 'qq', 'kk'])]

In [ ]:
# def extract_keywords_from_url(url):
#     url_parts = url.split('/')
#     keywords = ' '.join(url_parts[3:-1])
#     return keywords
# df_train['content'].fillna(df_train['url'].apply(extract_keywords_from_url), inplace=True)
df_train['content'] = df_train['content'].apply(lambda x: str(x)[:1465] if len(str(x)) > 1465 else str(x))
df_train = df_train.drop_duplicates(subset='url', keep='first')

In [ ]:
def summary(df):
    summary_df = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summary_df['missing#'] = df.isna().sum()
    summary_df['missing%'] = (df.isna().sum())/len(df)
    summary_df['uniques'] = df.nunique().values
    summary_df['count'] = df.count().values
    return summary_df

summary(df_train).style.background_gradient(cmap='Greens')

In [ ]:
def short_describe(df):
    rows, cols = df.shape
    col_names = ', '.join(df.columns.tolist())
    print(f'* Number of Rows: {rows}')
    print(f'* Number of Columns: {cols}')
    print(f'* Column names:\n {col_names}')
    
short_describe(df_train)

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

spacy_nlp = spacy.load('en_core_web_sm')

punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def spacy_tokenizer(sentence):
    sentence = re.sub('\'', '', sentence)
    sentence = re.sub('\w*\d\w*', '', sentence)
    sentence = re.sub(' +',' ',sentence)
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)
    sentence = re.sub(r'\n',' ',sentence)
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    tokens = spacy_nlp(sentence)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word)>2]
    return tokens

In [ ]:
print('Text Preprocessing & Tokenizing...')
%time df_train['content_tokenized'] = df_train['content'].map(lambda x: spacy_tokenizer(x))

In [ ]:
df_train.to_csv('tokenized_train.csv')

In [ ]:
df_train.head(10)

In [ ]:
content_tokenized = df_train['content_tokenized']
text = ' '. join(df_train['content_tokenized'].explode())
wordcloud = WordCloud(background_color='white').generate(text)
plt.figure(figsize=(15,15), facecolor = None, dpi=300)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Stop Words List
with open('/kaggle/input/nitec-epir-dataset/stopwords/stopwords-en.txt') as f:
    en_stoplist = f.read().splitlines()
with open('/kaggle/input/nitec-epir-dataset/stopwords/stopwords-ru.txt') as f:
    ru_stoplist = f.read().splitlines()
with open('/kaggle/input/nitec-epir-dataset/stopwords/stopwords-qq.txt') as f:
    qq_stoplist = f.read().splitlines()
with open('/kaggle/input/nitec-epir-dataset/stopwords/stopwords-kk.txt') as f:
    kk_stoplist = f.read().splitlines()

stoplist = en_stoplist + ru_stoplist + qq_stoplist + kk_stoplist

In [ ]:
from gensim import corpora

# Create Term Dictionary 
%time dictionary = corpora.Dictionary(content_tokenized)

stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)

In [ ]:
# Print top 50 items
dict_tokens = [[[dictionary[key], dictionary.token2id[dictionary[key]]] for key, value in dictionary.items() if key <= 50]]
print(dict_tokens)

In [ ]:
# Feature Extraction
corpus = [dictionary.doc2bow(desc) for desc in content_tokenized]
word_frequencies = [[(dictionary[id], frequency) for id, frequency in line] for line in corpus[0:3]]
print(word_frequencies)

In [ ]:
# Build TFIDF and LSI Models
%time tfidf_model = gensim.models.TfidfModel(corpus, id2word=dictionary)
%time lsi_model = gensim.models.LsiModel(tfidf_model[corpus], id2word=dictionary, num_topics=300)

In [ ]:
# Serialize and Store the corpus locallly for easy fetching whenever required
%time gensim.corpora.MmCorpus.serialize('tfidf_model', tfidf_model[corpus])
%time gensim.corpora.MmCorpus.serialize('lsi_model', lsi_model[tfidf_model[corpus]])

In [ ]:
# Load the indexed corpus
tfidf_corpus = gensim.corpora.MmCorpus('tfidf_model')
lsi_corpus = gensim.corpora.MmCorpus('lsi_model')

print(tfidf_corpus)
print(lsi_corpus)

In [ ]:
from gensim.similarities import MatrixSimilarity

%time index = MatrixSimilarity(lsi_corpus, num_features = lsi_corpus.num_terms)

In [ ]:
from operator import itemgetter

def search_answer(search_term):
    query_bow = dictionary.doc2bow(spacy_tokenizer(search_term))
    query_tfidf = tfidf_model[query_bow]
    query_lsi = lsi_model[query_tfidf]
    index.num_best = 5
    listx = index[query_lsi]
    listx.sort(key=itemgetter(1), reverse=True)
    ans = []
    for i, j in enumerate(listx):
        ans.append({
            'accuracy': round((j[1]*100),2),
            'id': df_train.iloc[j[0]]['id'],
            'url': df_train.iloc[j[0]]['url'],
            'category': df_train.iloc[j[0]]['category']
        })
        if i == (index.num_best-1):
            break
    return pd.DataFrame(ans, columns=['accuracy', 'id', 'url', 'category'])

In [ ]:
# Testing: Example Searching
ans = search_answer('Where can I find the latest updates and information about obtaining public services online in Kazakhstan?')
ans['url'].iloc[0]

In [ ]:
submission_df = pd.DataFrame(columns=['accuracy', 'id', 'url', 'category'])

for _, row in df_test.iterrows():
    result = search_answer(row['question']).iloc[[0]]
    result.rename(columns={'id': 'index'}, inplace=True)
    result['id'] = row['id']
    submission_df = pd.concat([submission_df, result], ignore_index=True)

submission_df = submission_df[['id', 'index']]
submission_df.to_csv('submission.csv', index=False)
submission_df.head(10)